# Team Gamma - CME Mini S&P (ES)

## Data Preparation

### Import Libraries

In [1]:
import glob
import gc
import numpy as np
import pandas as pd

%matplotlib inline

### Configuration

In [2]:
MULTIPLIER = 50
COMMISSION = 0.85
TRAIN = 'TRAIN'
TEST = 'TEST'
TRAIN_PICKLE = 'TRAIN_PICKLE.pickle'
TEST_PICKLE = 'TEST_PICKLE.pickle'
TRAIN_DIR = 'TRAIN DATA/*.csv'
TEST_DIR = 'TEST DATA/*.csv'

### Serialize Data

In [3]:
# Set to False if your serialized file has not yet been generated
# Remember to change back to "True" once the serialized file has been generated
DATA_SERIALIZED = True

if not DATA_SERIALIZED:
    
    # Writing files to pickle
    train_files = glob.glob(TRAIN_DIR)
    test_files = glob.glob(TEST_DIR)
    train_files.sort()
    test_files.sort()
    train_list = []
    test_list = []
    for filename in train_files:
        train_list.append(pd.read_csv(filename, index_col=0))
    for filename in test_files:
        test_list.append(pd.read_csv(filename, index_col=0))
    train = pd.concat(train_list)
    test = pd.concat(test_list)
    train.to_pickle(TRAIN_PICKLE)
    test.to_pickle(TEST_PICKLE)
    
    # Garbage collection
    del(train_files)
    del(test_files)
    del(train_list)
    del(test_list)
    del(train)
    del(test)
    gc.collect()


### Read Data

In [4]:
train_data = pd.read_pickle(TRAIN_PICKLE)
test_data = pd.read_pickle(TEST_PICKLE)
train_data.index = pd.to_datetime(train_data.index)
test_data.index = pd.to_datetime(test_data.index)

### Utility Functions

In [5]:
def filterData (data, columns):
    toReturn = data.loc[data['eB'] > 0, :] # Filter out no data rows
    return toReturn.loc[:, columns]

### Performance Measures

In [6]:
def winning_probability (pnls):
    if len(pnls) == 0:
        return np.nan
    positive = 0
    for pnl in pnls:
        if pnl > 0:
            positive += 1
    return positive / len(pnls)

def reward_to_risk_ratio (pnls):
    if len(pnls) == 0:
        return np.nan
    wins = []
    losses = []
    for pnl in pnls:
        if pnl > 0:
            wins.append(pnl)
        else:
            losses.append(pnl)
    return np.mean(wins) / abs(np.mean(losses))

def t_stat (pnls):
    if len(pnls) == 0:
        return np.nan
    return np.sqrt(len(pnls)) * (np.mean(pnls) / np.std(pnls))

## Order Flow Model (2)

### Data Preparation

In [7]:
OF2_COLUMNS = ['eB', 'eA', 'V', 'OF']

In [8]:
def init_of2_data (data):
    df = filterData(data, OF2_COLUMNS)
    df.rename(index=str, columns={'eB':'bid','eA':'ask','V':'volume','OF':'orderFlow'}, inplace=True)
    return df

### Parameters

In [9]:
TICK_SIZE = 0.25
STOPPING_LEVEL = [3, 4, 5, 6]
CLOSING_LEVEL = [1, 2, 3, 4, 5, 6, 7, 8]
THRESHOLD = [200, 400, 600, 800, 1000]

### Model

In [10]:
def of2_model (data, OF_LIMIT, STOP_LOSS_LEVEL, CLOSING_LEVEL):
    
    # Prepare initial data for manipulation
    df = init_of2_data(data)
    stopping_amount = STOP_LOSS_LEVEL * TICK_SIZE
    closing_amount = CLOSING_LEVEL * TICK_SIZE
    
    # Add signals [-1 : Sell, 0: None, 1: Buy]
    df['signal'] = df['orderFlow'].apply(lambda x : -1 if (abs(x) > OF_LIMIT and x > 0) else (1 if (abs(x) > OF_LIMIT and x < 0) else 0))

    # Add order signals [-1 : Sell, 0: None, 1: Buy]
    
    # Initialise order signals column
    df['order'] = 0
    i = 0 # counter
    while i < len(df):
    
        # Current row's signal
        curr_signal = df['signal'].iloc[i]

        # Check if current row's signal is not 0, else do nothing
        if curr_signal != 0:
            
            df['order'].iloc[i] = curr_signal
            
            if curr_signal > 0:
                curr_sig_type = 'bid'
                fwd_sig_type = 'ask'
            else:
                curr_sig_type = 'ask'
                fwd_sig_type = 'bid'
            
            curr_px = df[curr_sig_type].iloc[i]
            
            if curr_signal > 0:
                stop_px = curr_px - stopping_amount
                close_px = curr_px + closing_amount
            else:
                stop_px = curr_px + stopping_amount
                close_px = curr_px - closing_amount
            
            # Counter for forward tracking
            j = i + 1
            
            while j < len(df):
                
                forward_row = df.iloc[j, :]
                px = forward_row[fwd_sig_type]
                
                # Close position at the end of the day
#                 if j < len(df) - 1:
#                     if (pd.to_datetime(df.iloc[j + 1, :].name) - pd.to_datetime(forward_row.name)) > pd.Timedelta('5 min'):
#                         if curr_signal > 0:
#                             df['order'].iloc[j] = -1
#                         else:
#                             df['order'].iloc[j] = 1
#                         j += 1
#                         break
#                 else:
#                     if curr_signal > 0:
#                         df['order'].iloc[j] = -1
#                     else:
#                         df['order'].iloc[j] = 1
#                     j += 1
#                     break
                    
                # Long position
                if curr_signal > 0:
                    # Check if past closing position level
                    if px >= close_px:
                        df['order'].iloc[j] = -1
                        break
                    elif px <= stop_px:
                        df['order'].iloc[j] = -1
                        break
                    else:
                        j += 1
                    
                # Short position
                else:
                    # Check if past closing position level
                    if px <= close_px:
                        df['order'].iloc[j] = 1
                        break
                    elif px >= stop_px:
                        df['order'].iloc[j] = 1
                        break
                    else:
                        j += 1
            i = j + 1
                    
        else:
            i += 1
        
    df['transaction'] = df.apply(lambda x: x['bid'] * -1 if x['order'] > 0 else (x['ask'] if x['order'] < 0 else 0), axis=1)
    
    return df

### Backtest

In [11]:
of2_results_df = pd.DataFrame(columns=['Stopping Loss Level', 'Closing Level', 'Threshold', 'Winning Probability', 'Reward to Risk', 'T-Stat'])

In [12]:
for of_limit in THRESHOLD:
    for stop in STOPPING_LEVEL:
        for close in CLOSING_LEVEL:
            print(f'Stoploss: { stop } | Closing: { close } | Threshold: { of_limit }')
            transactions = of2_model(train_data, of_limit, stop, close)['transaction']
            positions = [x for x in transactions if x != 0]
            pnl = []
            for i in range(0, len(positions), 2):
                pos = (positions[i] + positions[i + 1]) * MULTIPLIER - 2 * COMMISSION
                pnl.append(pos)
            wp = winning_probability(pnl)
            rr = reward_to_risk_ratio(pnl)
            ts = t_stat(pnl)
            of2_results_df.loc[len(of2_results_df)] = [stop, close, of_limit, wp, rr, ts]

Stoploss: 3 | Closing: 1 | Threshold: 200


/Users/louis/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Stoploss: 3 | Closing: 2 | Threshold: 200
Stoploss: 3 | Closing: 3 | Threshold: 200
Stoploss: 3 | Closing: 4 | Threshold: 200
Stoploss: 4 | Closing: 1 | Threshold: 200
Stoploss: 4 | Closing: 2 | Threshold: 200
Stoploss: 4 | Closing: 3 | Threshold: 200
Stoploss: 4 | Closing: 4 | Threshold: 200
Stoploss: 5 | Closing: 1 | Threshold: 200
Stoploss: 5 | Closing: 2 | Threshold: 200
Stoploss: 5 | Closing: 3 | Threshold: 200
Stoploss: 5 | Closing: 4 | Threshold: 200
Stoploss: 6 | Closing: 1 | Threshold: 200
Stoploss: 6 | Closing: 2 | Threshold: 200
Stoploss: 6 | Closing: 3 | Threshold: 200
Stoploss: 6 | Closing: 4 | Threshold: 200
Stoploss: 3 | Closing: 1 | Threshold: 400
Stoploss: 3 | Closing: 2 | Threshold: 400
Stoploss: 3 | Closing: 3 | Threshold: 400
Stoploss: 3 | Closing: 4 | Threshold: 400
Stoploss: 4 | Closing: 1 | Threshold: 400
Stoploss: 4 | Closing: 2 | Threshold: 400
Stoploss: 4 | Closing: 3 | Threshold: 400
Stoploss: 4 | Closing: 4 | Threshold: 400
Stoploss: 5 | Closing: 1 | Thresho

In [13]:
of2_results_df.to_csv('OF2_MODEL_RESULTS_APPENDED.csv', index=False)

### Test Set Performance

In [16]:
# Based on top T-statistic performance

# Change with optimised training results
of_limit = 200
stop = 3
close = 1

transactions = of2_model(test_data, of_limit, stop, close)['transaction']
positions = [x for x in transactions if x != 0]
pnl = []
for i in range(0, len(positions), 2):
    pos = (positions[i] + positions[i + 1]) * MULTIPLIER - 2 * COMMISSION
    pnl.append(pos)
wp = winning_probability(pnl)
rr = reward_to_risk_ratio(pnl)
ts = t_stat(pnl)

print(f'PNL: { sum(pnl) } | Number of Closed Positions: { len(pnl) } | Winning Probability: { wp } | Reward to Risk Ratio: { rr } | T-Statistic: { ts }')

/Users/louis/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


PNL: 13103.699999999886 | Number of Closed Positions: 939 | Winning Probability: 0.9403620873269436 | Reward to Risk Ratio: 0.34558732261288083 | T-Statistic: 12.71696214669417


In [17]:
# Based on top Reward-to-Risk Ratio with at least T-stat of 2

# Change with optimised training results
of_limit = 200
stop = 3
close = 8

transactions = of2_model(test_data, of_limit, stop, close)['transaction']
positions = [x for x in transactions if x != 0]
pnl = []
for i in range(0, len(positions), 2):
    pos = (positions[i] + positions[i + 1]) * MULTIPLIER - 2 * COMMISSION
    pnl.append(pos)
wp = winning_probability(pnl)
rr = reward_to_risk_ratio(pnl)
ts = t_stat(pnl)

print(f'PNL: { sum(pnl) } | Number of Closed Positions: { len(pnl) } | Winning Probability: { wp } | Reward to Risk Ratio: { rr } | T-Statistic: { ts }')

/Users/louis/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


PNL: 7454.300000000053 | Number of Closed Positions: 321 | Winning Probability: 0.40809968847352024 | Reward to Risk Ratio: 2.25733646015299 | T-Statistic: 2.1902812757391104


In [18]:
# Based on top Winning Probability with at least T-stat of 2

# Change with optimised training results
of_limit = 200
stop = 6
close = 1

transactions = of2_model(test_data, of_limit, stop, close)['transaction']
positions = [x for x in transactions if x != 0]
pnl = []
for i in range(0, len(positions), 2):
    pos = (positions[i] + positions[i + 1]) * MULTIPLIER - 2 * COMMISSION
    pnl.append(pos)
wp = winning_probability(pnl)
rr = reward_to_risk_ratio(pnl)
ts = t_stat(pnl)

print(f'PNL: { sum(pnl) } | Number of Closed Positions: { len(pnl) } | Winning Probability: { wp } | Reward to Risk Ratio: { rr } | T-Statistic: { ts }')

/Users/louis/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


PNL: 12182.399999999936 | Number of Closed Positions: 878 | Winning Probability: 0.9635535307517085 | Reward to Risk Ratio: 0.16792472312378987 | T-Statistic: 9.542513039720875
